## Scraping Data from CodeClimate


In [25]:
import requests
import json
import os
from dotenv import load_dotenv
import datetime
import subprocess as sp
import pandas as pd
from collections import defaultdict

In [26]:
load_dotenv()

True

In [27]:
github_token = os.getenv("GITHUB_ACCESS_TOKEN")
sonarqube_token = os.getenv("SONARQUBE_ACCESS_TOKEN_VUE")
sonarqube_server_url = os.getenv("SONARQUBE_SERVER_URL")

In [28]:
demo_github_slug = "vuejs"
demo_repo_name = "vue"
demo_repo_path = "C:/SWE Class/Github Desktop/TestProjects/vue"

In [29]:
sonarqube_token, sonarqube_server_url

('sqp_6a56b3fba4ce9fb8a298c75695285615a47d87dc', 'http://localhost:9000')

## Git CLI Data Extraction

In [30]:
def get_tag_list(repo_path: str, max_output: int = 20) -> list[str]:
    # Subprocess to get the git directory
    sp.run(
        ["git", "rev-parse", "--git-dir"],
        cwd=repo_path,
        capture_output=True,
        check=True,
    )

    # Subprocess to get the git tags in the repo
    result = sp.run(
        ["git", "tag", "-l"],
        cwd=repo_path,
        capture_output=True,
        check=True,
    )

    tag_list = result.stdout.decode("utf-8").split("\n")
    tag_list = [tag for tag in tag_list if tag != ""]

    if max_output is not None and len(tag_list) > max_output:
        tag_list = tag_list[-max_output:]

    return tag_list

In [31]:
get_tag_list(demo_repo_path, None)

['0.10.0-rc',
 '0.11.0',
 '0.11.0-rc',
 '0.11.0-rc2',
 '0.11.0-rc3',
 '0.11.1',
 '0.11.10',
 '0.11.2',
 '0.11.3',
 '0.11.4',
 '0.11.5',
 '0.11.6',
 '0.11.7',
 '0.11.8',
 '0.11.9',
 '0.12.0',
 '0.12.0-beta2',
 '0.12.0-beta3',
 '0.12.0-beta4',
 '0.12.0-beta5',
 '0.12.0-csp',
 '0.12.0-rc',
 '0.12.0-rc2',
 '0.12.1',
 '0.12.1-csp',
 '0.12.1-csp.1',
 '0.12.1-csp.2',
 '0.12.10',
 '0.12.10-csp',
 '0.12.11',
 '0.12.11-csp',
 '0.12.12',
 '0.12.12-csp',
 '0.12.13',
 '0.12.14',
 '0.12.14-csp',
 '0.12.15',
 '0.12.15-csp',
 '0.12.16',
 '0.12.16-csp',
 '0.12.2',
 '0.12.3',
 '0.12.4',
 '0.12.5',
 '0.12.6',
 '0.12.6-csp',
 '0.12.7',
 '0.12.7-csp',
 '0.12.8',
 '0.12.8-csp',
 '0.12.9',
 '0.12.9-csp',
 '0.6.0',
 '1.0.0',
 '1.0.0-alpha.1',
 '1.0.0-alpha.2',
 '1.0.0-alpha.3',
 '1.0.0-alpha.4',
 '1.0.0-alpha.5',
 '1.0.0-alpha.6',
 '1.0.0-alpha.7',
 '1.0.0-alpha.8',
 '1.0.0-beta.1',
 '1.0.0-beta.2',
 '1.0.0-beta.3',
 '1.0.0-beta.4',
 '1.0.0-csp',
 '1.0.0-migration',
 '1.0.0-rc.1',
 '1.0.0-rc.2',
 '1.0.0-rc.2-

In [32]:
def get_tag_and_timestamp(repo_path: str) -> list[tuple[str, str, int]]:
    tag_list = get_tag_list(repo_path, None)
    tag_and_timestamp = []

    for tag in tag_list:
        result = sp.run(
            ["git", "log", "-1", "--format=%cd %ct", "--date=short", tag],
            cwd=repo_path,
            capture_output=True,
            check=True,
            text=True,
        )

        date, timestamp = result.stdout.strip().split()
        tag_and_timestamp.append((tag, date, int(timestamp)))
        # print(result.stdout.strip())
    return tag_and_timestamp

In [33]:
tags_and_timestamps = get_tag_and_timestamp(demo_repo_path)
tags_and_timestamps

[('0.10.0-rc', '2014-03-19', 1395263823),
 ('0.11.0', '2014-11-06', 1415325162),
 ('0.11.0-rc', '2014-09-26', 1411781027),
 ('0.11.0-rc2', '2014-10-07', 1412701646),
 ('0.11.0-rc3', '2014-10-24', 1414123890),
 ('0.11.1', '2014-11-30', 1417393461),
 ('0.11.10', '2015-05-07', 1431025236),
 ('0.11.2', '2014-12-01', 1417487864),
 ('0.11.3', '2014-12-02', 1417540925),
 ('0.11.4', '2014-12-07', 1417984981),
 ('0.11.5', '2015-02-05', 1423171588),
 ('0.11.6', '2015-04-18', 1429341079),
 ('0.11.7', '2015-04-20', 1429586386),
 ('0.11.8', '2015-04-21', 1429647040),
 ('0.11.9', '2015-05-06', 1430944803),
 ('0.12.0', '2015-06-12', 1434130469),
 ('0.12.0-beta2', '2015-05-16', 1431798762),
 ('0.12.0-beta3', '2015-05-22', 1432325270),
 ('0.12.0-beta4', '2015-05-26', 1432653677),
 ('0.12.0-beta5', '2015-05-29', 1432952850),
 ('0.12.0-csp', '2015-06-12', 1434142186),
 ('0.12.0-rc', '2015-06-01', 1433192205),
 ('0.12.0-rc2', '2015-06-04', 1433444419),
 ('0.12.1', '2015-06-14', 1434259685),
 ('0.12.1-csp'

In [34]:
# Checkout to a specific tag and show the output of the operation
def checkout_to_tag(repo_path: str, tag: str):
    result = sp.run(
        ["git", "checkout", tag],
        cwd=repo_path,
        capture_output=True,
        check=True,
        text=True,
    )
    print(result.stdout.strip())

In [35]:
# Checkout to a specific tag use sonar-scanner to scan the project
def sonar_scan(repo_path: str, tag: str):
    result = sp.run(
        [
            "sonar-scanner",
            "-D",
            f"sonar.projectBaseDir={demo_repo_path}",
            "-D",
            "sonar.projectVersion=" + tag,
            "-D",
            f"sonar.projectKey={demo_repo_name}",
            "-D",
            "sonar.sources=.",
            "-D",
            f"sonar.host.url={sonarqube_server_url}",
            "-D",
            "sonar.login=" + sonarqube_token,
        ],
        cwd=repo_path,
        capture_output=True,
        check=True,
        text=True,
        shell=True,
    )
    # print(result.stdout)

In [36]:
# Call the sonarqube api to get the metrics of the project
def get_metrics() -> dict:
    result = sp.run(
        [
            "curl.exe",
            f"{sonarqube_server_url}/api/measures/component?component={demo_repo_name}&metricKeys=bugs,vulnerabilities,code_smells,coverage,duplicated_lines_density,violations,ncloc,security_rating,reliability_rating,sqale_rating",
        ],
        capture_output=True,
        check=True,
        text=True,
        # shell=True,
    )
    return json.loads(result.stdout)

'code_smells',
 'sqale_rating',
 'vulnerabilities',
 'security_rating',
 'duplicated_lines_density',
 'violations',
 'bugs',
 'reliability_rating',
 'coverage',
 'ncloc'

In [37]:
metrics_dict = {
    "version": [],
    "date": [],
    "timestamp": [],
    "bugs": [],
    "vulnerabilities": [],
    "code_smells": [],
    "coverage": [],
    "duplicated_lines_density": [],
    "violations": [],
    "ncloc": [],
    "security_rating": [],
    "reliability_rating": [],
    "sqale_rating": [],
}

In [38]:
# Get metrics of the project
def get_metrics_from_version_tags(
    repo_path: str, tag: str, date: str, timestamp: int, output_dict=metrics_dict
):
    checkout_to_tag(repo_path, tag)
    sonar_scan(repo_path, tag)
    metrics_result = get_metrics()

    output_dict["version"].append(tag)
    output_dict["date"].append(date)
    output_dict["timestamp"].append(timestamp)

    for metric in metrics_result["component"]["measures"]:
        output_dict[metric["metric"]].append(metric["value"])

    return output_dict

In [39]:
# A mock call to test sonar-scanner
sp.run(
    ["sonar-scanner", "-h"],
    capture_output=True,
    check=True,
    text=True,
    shell=True,
)

CompletedProcess(args=['sonar-scanner', '-h'], returncode=0, stdout='INFO: \nINFO: usage: sonar-scanner [options]\nINFO: \nINFO: Options:\nINFO:  -D,--define <arg>     Define property\nINFO:  -h,--help             Display help information\nINFO:  -v,--version          Display version information\nINFO:  -X,--debug            Produce execution debug output\n', stderr='')

In [40]:
# for tag_and_timestamp in tags_and_timestamps:
#     get_metrics_from_version_tags(demo_repo_path, *tag_and_timestamp)

In [41]:
# metrics_dict

In [42]:
# metrics_dataset = pd.DataFrame(metrics_dict, index=None)
# metrics_dataset

In [43]:
# metrics_dataset.to_csv("../Datasets/vue_metrics.csv", index=False)

## Issues Narrowing from SonarQube

In [44]:
# A function that can convert given time string into minutes. The time string may include hours, minutes and seconds. It is in the following format: "1d2h3min". The days, hours or minutes may be missing. If the string is empty, return 0.


def time_str_to_minutes(time: str) -> int:
    if time == "":
        return 0

    time = time.replace("d", " ").replace("h", " ").replace("min", " ").split()
    time = [int(t) for t in time]

    if len(time) == 1:
        return time[0]
    elif len(time) == 2:
        return time[0] * 60 + time[1]
    elif len(time) == 3:
        return time[0] * 24 * 60 + time[1] * 60 + time[2]
    else:
        raise ValueError("Invalid time string")

In [45]:
clean_code_attribute_categories = [
    "RESPONSIBLE",
    "ADAPTABLE",
    "CONSISTENT",
    "INTENTIONAL",
]

severities = ["LOW", "MEDIUM", "HIGH"]

quality_attributes = [
    "RELIABILITY",
    "SECURITY",
    "MAINTAINABILITY",
]

In [46]:
clean_code_attribute_dict = defaultdict()

for category in clean_code_attribute_categories:
    clean_code_attribute_dict[category] = defaultdict(list)

clean_code_attribute_dict

defaultdict(None,
            {'RESPONSIBLE': defaultdict(list, {}),
             'ADAPTABLE': defaultdict(list, {}),
             'CONSISTENT': defaultdict(list, {}),
             'INTENTIONAL': defaultdict(list, {})})

In [47]:
def get_quality_issue_values_from_api(clean_code_category: str, severity: str):
    """
    return a tuple of (total_security_issues, total_reliability_issues, total_maintainability_issues, total_debt)
    """

    result = sp.run(
        [
            "curl.exe",
            f"{sonarqube_server_url}/api/issues/search?cleanCodeAttributeCategories={clean_code_category}",
        ],
        capture_output=True,
        check=True,
        text=True,
        # shell=True,
    )

    issues_list = json.loads(result.stdout)["issues"]

    # Get the software quality, maintainability and debt from the issues list
    filtered_issues_list = [
        {
            "software_quality_category": issue["impacts"][0]["softwareQuality"],
            "severity": issue["impacts"][0]["severity"],
            "debt": time_str_to_minutes(issue["debt"]),
        }
        for issue in issues_list
        if issue["type"] == "CODE_SMELL" and issue["impacts"][0]["severity"] == severity
    ]

    # Get the total number of security, reliability and maintainability quality issue count and debt
    total_security_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "SECURITY"
        ]
    )
    total_reliability_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "RELIABILITY"
        ]
    )
    total_maintainability_issues = len(
        [
            issue
            for issue in filtered_issues_list
            if issue["software_quality_category"] == "MAINTAINABILITY"
        ]
    )

    total_debt = sum([issue["debt"] for issue in filtered_issues_list])

    return (
        total_security_issues,
        total_reliability_issues,
        total_maintainability_issues,
        total_debt,
    )

In [48]:
# For each version, get the total number of security, reliability and maintainability quality issue count and debt and store them in the corrsponding dictionary based on the clean code category

for tag_and_timestamp in tags_and_timestamps:
    tag, date, timestamp = tag_and_timestamp
    checkout_to_tag(demo_repo_path, tag)
    sonar_scan(demo_repo_path, tag)

    for clean_code_category in clean_code_attribute_categories:
        clean_code_attribute_dict[clean_code_category]["version"].append(tag)
        clean_code_attribute_dict[clean_code_category]["date"].append(date)
        clean_code_attribute_dict[clean_code_category]["timestamp"].append(timestamp)

        for severity in severities:
            (
                total_security_issues,
                total_reliability_issues,
                total_maintainability_issues,
                total_debt,
            ) = get_quality_issue_values_from_api(clean_code_category, severity)

            clean_code_attribute_dict[clean_code_category][
                f"security_issues_{severity.lower()}"
            ].append(total_security_issues)

            clean_code_attribute_dict[clean_code_category][
                f"reliability_issues_{severity.lower()}"
            ].append(total_reliability_issues)

            clean_code_attribute_dict[clean_code_category][
                f"maintainability_issues_{severity.lower()}"
            ].append(total_maintainability_issues)

            clean_code_attribute_dict[clean_code_category][
                f"total_debt_{severity.lower()}"
            ].append(total_debt)

In [49]:
for clean_code_attribute, attribute_dict in clean_code_attribute_dict.items():
    dataframe = pd.DataFrame(attribute_dict, index=None)
    dataframe.to_csv(
        f"../Datasets/{demo_repo_name}/{demo_repo_name}_{clean_code_attribute.lower()}_issues.csv",
        index=False,
    )